Calculates sea level pressure global means and standard deviations

Variables used: psl

In [1]:
import os
import xarray as xr 
import numpy as np
import pandas as pd
import functions as func

In [ ]:
SCALE_TIMESERIES =  os.environ.get('SCALE_TIMESERIES')
CREATE_GRAPHICS = os.environ.get('CREATE_GRAPHICS')
PNG_SCALE = float(os.environ.get('PNG_SCALE'))
OUTPUT_TYPE = os.environ.get('OUTPUT_TYPE')
COLORMAP = os.environ.get('COLORMAP')
OUTDIR = os.environ.get('OUTDIR')
PNG_SCALE_SUMMARY = float(os.environ.get('PNG_SCALE_SUMMARY'))

In [ ]:
vname = ("PSL","psl","slp","SLP","prmsl","msl","slp_dyn")

names = []
paths = []
syear = []
eyear = []
names_EM = []
EM_num = []
with open('../namelist_byvar/namelist_psl') as f:
  for line in f:
#    print(line)
    check = line.split(' | ')
#    print(check[0])
    names.append(check[0])
    paths.append(check[1])
    syear.append(check[2])
    eyear.append(check[3])
    ems = check[4].split('-')
    EM_num.append(ems[0])
    names_EM.append(ems[1])
    
gg = -1
for mr in paths:
  gg = gg+1
  print(type(mr))
  fil0 = !ls {mr}   # this works as direct input into open_mfdataset!
  tfiles_str = fil0.nlstr    # convert to string
  tfiles = tfiles_str.split("\n")  # split string to list 
#  print(type(tfiles)) 
  cpathS = tfiles[0]
  cpathE = tfiles[len(tfiles)-1]
  sydata = int(cpathS[len(cpathS)-16:len(cpathS)-12])  # start year of data (specified in file name)
  smdata = int(cpathS[len(cpathS)-12:len(cpathS)-10])  # start month of data
  eydata = int(cpathE[len(cpathE)-9:len(cpathE)-5])    # end year of data 
  emdata = int(cpathE[len(cpathE)-5:len(cpathE)-3])    # end month of data

  fno = names[gg]+'.cvdp_data.psl.mean_stddev.py.'+syear[gg]+'-'+eyear[gg]+'.nc'

  ds = xr.open_mfdataset(fil0,decode_times=False) 
  for v in vname:
    if v in ds:
      arr = ds.data_vars[v]
#      print(arr)
      break
    
  try:
    tarr = arr
  except NameError:
    print('Variable psl not found within file '+names[gg])
    arr = func.create_empty_array( int(syear[gg]), int(eyear[gg]), 1, 12, 'time_lat_lon')
    
  nd = arr.ndim
  if (nd <= 2):   # (needs testing)
    print('Whoa, less than 3 dimensions, curvilinear data is not currently allowed')
    arr = func.create_empty_array( int(syear[gg]), int(eyear[gg]), 1, 12, 'time_lat_lon')

  if '_FillValue' not in arr.attrs:   # assign _FillValue if one is not present
    if 'missing_value' in arr.attrs:
      arr.attrs['_FillValue'] = arr.attrs['missing_value']
    else:
      arr.attrs['_FillValue'] = 1.e20

  if [True for dimsize in arr.sizes if dimsize == 1]:
    arr = arr.squeeze()

  dsc = arr.dims   # rename dimensions
  arr = arr.rename({dsc[0] : 'time'})
  arr = arr.rename({dsc[1] : 'lat'})
  arr = arr.rename({dsc[2] : 'lon'})
     
  if arr.coords['lat'][0] >= 0:
    arr = arr[:, ::-1, :]   # flip the latitudes
    
  if (arr.lon[0] < 0):     # Isla's method to alter lons to go from -180:180->0:360  (needs testing)
    print("flipping longitudes")
    arr.coords['lon'] = (arr.coords['lon'] + 360) % 360
    arr = arr.sortby(arr.lon)  
    
    
  if int(syear[gg]) < sydata or int(eyear[gg]) > eydata:   # check that the data file falls within the specified data analysis range
    arr = func.create_empty_array( int(syear[gg]), int(eyear[gg]), 1, 12, 'time_lat_lon')
  else:
    timeT = func.yyyymm_time(sydata, eydata, "integer")    # reassign time coordinate to YYYYMM
    time = timeT.sel(time=slice(sydata*100+smdata, eydata*100+emdata))
    arr = arr.assign_coords(time=time) 
    arr = arr.sel(time=slice(int(syear[gg])*100+1,int(eyear[gg])*100+12))
    
#  print(arr['time'])  
    
  mocheck = np.array([((int(syear[gg])*100+1)-min(arr.coords['time'])), ((int(eyear[gg])*100+12) - max(arr.coords['time']))])
  if [True for mon in mocheck if mon != 0]:  
    if mocheck[0] != 0:
      print("First requested year is incomplete")
    if mocheck[1] != 0:
      print("Last requested year is incomplete")
      print(f'Incomplete data year(s) requested for file {paths[gg]}, printing out time and creating blank array')
      print(f'Time requested: {syear[gg]}-{eyear[gg]}')
      print(arr.coords['time'])
      arr = func.create_empty_array(int(syear[gg]), int(eyear[gg]), 1, 12, 'time_lat_lon')

    
  time2 = func.YYYYMM2date(arr.time.values,'standard')   #switch from YYYYMM->datetime64
  arr = arr.assign_coords(time=time2)
    
  if (arr.units == 'Pa'):
    arr = arr/100.
    arr.attrs = {'units':'hPa'}    

  smean4 = arr.groupby('time.season').mean('time')
  temp = arr.where( (arr['time.month'] == 7) | (arr['time.month'] == 8 | (arr['time.month'] == 9) ))
  smeanJAS = temp.mean('time')      
  temp = arr.where( (arr['time.month'] == 1) | (arr['time.month'] == 2 | (arr['time.month'] == 3) ))
  smeanJFM = temp.mean('time') 
                   
  sstd4 = arr.groupby('time.season').std('time')
  temp = arr.where( (arr['time.month'] == 7) | (arr['time.month'] == 8 | (arr['time.month'] == 9) ))
  sstdJAS = temp.std('time')      
  temp = arr.where( (arr['time.month'] == 1) | (arr['time.month'] == 2 | (arr['time.month'] == 3) ))
  sstdJFM = temp.std('time') 

  arr_roll = arr.rolling(time=12, center=True).mean()
  arr12 = arr_roll[6::12,:,:]
  smeanANN = arr12.mean('time')
  sstdANN = arr12.std('time')
    
  smeanDJF = smean4[0,:,:]
  smeanDJF = smeanDJF.rename("psl_spatialmean_djf")
  smeanDJF.attrs = {'units':'hPa','long_name':'psl climatology (DJF)'}    
  smeanMAM = smean4[1,:,:]
  smeanMAM = smeanMAM.rename("psl_spatialmean_mam")
  smeanMAM.attrs = {'units':'hPa','long_name':'psl climatology (MAM)'}
  smeanJJA = smean4[2,:,:]
  smeanJJA = smeanJJA.rename("psl_spatialmean_jja")
  smeanJJA.attrs = {'units':'hPa','long_name':'psl climatology (JJA)'}
  smeanSON = smean4[3,:,:]
  smeanSON = smeanSON.rename("psl_spatialmean_son")
  smeanSON.attrs = {'units':'hPa','long_name':'psl climatology (SON)'}
  smeanJFM = smeanJFM.rename("psl_spatialmean_jfm")
  smeanJFM.attrs = {'units':'hPa', 'long_name':'psl climatology (JFM)'}
  smeanJAS = smeanJJA.rename("psl_spatialmean_jas")
  smeanJAS.attrs = {'units':'hPa', 'long_name':'psl climatology (JAS)'}
  smeanANN = smeanANN.rename("psl_spatialmean_ann")
  smeanANN.attrs = {'units':'hPa', 'long_name':'psl climatology (annual)'}
    
  sstdDJF = sstd4[0,:,:]
  sstdDJF = sstdDJF.rename("psl_spatialstddev_djf")    
  sstdDJF.attrs = {'units':'hPa','long_name':'psl standard deviation (DJF)'}
  sstdMAM = sstd4[1,:,:]
  sstdMAM = sstdMAM.rename("psl_spatialstddev_mam")
  sstdMAM.attrs = {'units':'hPa','long_name':'psl standard deviation (MAM)'}
  sstdJJA = sstd4[2,:,:]
  sstdJJA = sstdJJA.rename("psl_spatialstddev_jja")
  sstdJJA.attrs = {'units':'hPa','long_name':'psl standard deviation (JJA)'}
  sstdSON = sstd4[3,:,:]
  sstdSON = sstdSON.rename("psl_spatialstddev_son")
  sstdSON.attrs = {'units':'hPa','long_name':'psl standard deviation (SON)'}
  sstdJFM = sstdJFM.rename("psl_spatialstddev_jfm")
  sstdJFM.attrs = {'units':'hPa','long_name':'psl standard deviation (JFM)'}
  sstdJAS = sstdJAS.rename("psl_spatialstddev_jas")
  sstdJAS.attrs = {'units':'hPa','long_name':'psl standard deviation (JAS)'}
  sstdANN = sstdANN.rename("psl_spatialstddev_ann")
  sstdANN.attrs = {'units':'hPa','long_name':'psl standard deviation (annual)'}
#  print(smeanDJF.name)
#  print(smeanDJF)
# del(arr)    

  smeanDJF.to_netcdf(fno,encoding={'psl_spatialmean_djf':{'dtype':'float32', '_FillValue': 1.e20}, 'lat':{'dtype':'float32','_FillValue': None}, 'lon':{'dtype':'float32','_FillValue': None}})
  smeanMAM.to_netcdf(fno,encoding={'psl_spatialmean_mam':{'dtype':'float32', '_FillValue': 1.e20}},mode='a')
  smeanJJA.to_netcdf(fno,encoding={'psl_spatialmean_jja':{'dtype':'float32', '_FillValue': 1.e20}},mode='a')
  smeanSON.to_netcdf(fno,encoding={'psl_spatialmean_son':{'dtype':'float32', '_FillValue': 1.e20}},mode='a')
  smeanJFM.to_netcdf(fno,encoding={'psl_spatialmean_jfm':{'dtype':'float32', '_FillValue': 1.e20}},mode='a')
  smeanJAS.to_netcdf(fno,encoding={'psl_spatialmean_jas':{'dtype':'float32', '_FillValue': 1.e20}},mode='a')
  smeanANN.to_netcdf(fno,encoding={'psl_spatialmean_ann':{'dtype':'float32', '_FillValue': 1.e20}},mode='a')

  print(sstdDJF)
  sstdDJF.to_netcdf(fno,encoding={'psl_spatialstddev_djf':{'dtype':'float32', '_FillValue': 1.e20}},mode='a')
  sstdMAM.to_netcdf(fno,encoding={'psl_spatialstddev_mam':{'dtype':'float32', '_FillValue': 1.e20}},mode='a')
  sstdJJA.to_netcdf(fno,encoding={'psl_spatialstddev_jja':{'dtype':'float32', '_FillValue': 1.e20}},mode='a')
  sstdSON.to_netcdf(fno,encoding={'psl_spatialstddev_son':{'dtype':'float32', '_FillValue': 1.e20}},mode='a')
  sstdJFM.to_netcdf(fno,encoding={'psl_spatialstddev_jfm':{'dtype':'float32', '_FillValue': 1.e20}},mode='a')
  sstdJAS.to_netcdf(fno,encoding={'psl_spatialstddev_jas':{'dtype':'float32', '_FillValue': 1.e20}},mode='a')
  sstdANN.to_netcdf(fno,encoding={'psl_spatialstddev_ann':{'dtype':'float32', '_FillValue': 1.e20}},mode='a')
#  cdir = os.getcwd()
#  !ncatted -O -a source,global,c,c,{cdir}"/butterworth_filter_lowpass.annseas.apply.ipynb" {fno}  

  break
    

In [ ]:
print(sstdANN)
sstdANN.plot()

In [3]:
names = []
paths = []
syear = []
eyear = []
names_EM = []
EM_num = []

vn = 'psl'

with open('../namelist_byvar/namelist_'+vn) as f:
    for line in f:
        check = line.split(' | ')
        names.append(check[0])
        paths.append(check[1])
        syear.append(check[2])
        eyear.append(check[3])
        ems = check[4].split('-')
        EM_num.append(ems[0])
        names_EM.append(ems[1])
    
gg = -1
for mr in paths:
    gg = gg+1    

    fno = names[gg]+'.cvdp_data.'+vn+'.mean_stddev.py.'+syear[gg]+'-'+eyear[gg]+'.nc'
    fils = !ls {mr}   # this works as direct input into open_mfdataset!
    print(fils)
    farr = func.data_read_in_3D(fils,syear[gg],eyear[gg],vn)
    finarr = func.seasonal_mean_stddev(farr,vn)
    
    finarr[vn+'_spatialmean_djf'].to_netcdf(fno,encoding={vn+'_spatialmean_djf':{'dtype':'float32', '_FillValue': 1.e20}, 'lat':{'dtype':'float32','_FillValue': None}, 'lon':{'dtype':'float32','_FillValue': None}})
    finarr[vn+'_spatialmean_mam'].to_netcdf(fno,encoding={vn+'_spatialmean_mam':{'dtype':'float32', '_FillValue': 1.e20}},mode='a')
    finarr[vn+'_spatialmean_jja'].to_netcdf(fno,encoding={vn+'_spatialmean_jja':{'dtype':'float32', '_FillValue': 1.e20}},mode='a')
    finarr[vn+'_spatialmean_son'].to_netcdf(fno,encoding={vn+'_spatialmean_son':{'dtype':'float32', '_FillValue': 1.e20}},mode='a')
    finarr[vn+'_spatialmean_jfm'].to_netcdf(fno,encoding={vn+'_spatialmean_jfm':{'dtype':'float32', '_FillValue': 1.e20}},mode='a')
    finarr[vn+'_spatialmean_jas'].to_netcdf(fno,encoding={vn+'_spatialmean_jas':{'dtype':'float32', '_FillValue': 1.e20}},mode='a')
    finarr[vn+'_spatialmean_ann'].to_netcdf(fno,encoding={vn+'_spatialmean_ann':{'dtype':'float32', '_FillValue': 1.e20}},mode='a')
    finarr[vn+'_spatialstddev_djf'].to_netcdf(fno,encoding={vn+'_spatialstddev_djf':{'dtype':'float32', '_FillValue': 1.e20}},mode='a')
    finarr[vn+'_spatialstddev_mam'].to_netcdf(fno,encoding={vn+'_spatialstddev_mam':{'dtype':'float32', '_FillValue': 1.e20}},mode='a')
    finarr[vn+'_spatialstddev_jja'].to_netcdf(fno,encoding={vn+'_spatialstddev_jja':{'dtype':'float32', '_FillValue': 1.e20}},mode='a')
    finarr[vn+'_spatialstddev_son'].to_netcdf(fno,encoding={vn+'_spatialstddev_son':{'dtype':'float32', '_FillValue': 1.e20}},mode='a')
    finarr[vn+'_spatialstddev_jfm'].to_netcdf(fno,encoding={vn+'_spatialstddev_jfm':{'dtype':'float32', '_FillValue': 1.e20}},mode='a')
    finarr[vn+'_spatialstddev_jas'].to_netcdf(fno,encoding={vn+'_spatialstddev_jas':{'dtype':'float32', '_FillValue': 1.e20}},mode='a')
    finarr[vn+'_spatialstddev_ann'].to_netcdf(fno,encoding={vn+'_spatialstddev_ann':{'dtype':'float32', '_FillValue': 1.e20}},mode='a')
    del(finarr)
#    break

['/project/mojave/cesm2/LENS/atm/month_1/PSL/b.e21.BHISTsmbb.f09_g17.LE2-1301.020.cam.h0.PSL.185001-185912.nc', '/project/mojave/cesm2/LENS/atm/month_1/PSL/b.e21.BHISTsmbb.f09_g17.LE2-1301.020.cam.h0.PSL.186001-186912.nc', '/project/mojave/cesm2/LENS/atm/month_1/PSL/b.e21.BHISTsmbb.f09_g17.LE2-1301.020.cam.h0.PSL.187001-187912.nc', '/project/mojave/cesm2/LENS/atm/month_1/PSL/b.e21.BHISTsmbb.f09_g17.LE2-1301.020.cam.h0.PSL.188001-188912.nc', '/project/mojave/cesm2/LENS/atm/month_1/PSL/b.e21.BHISTsmbb.f09_g17.LE2-1301.020.cam.h0.PSL.189001-189912.nc', '/project/mojave/cesm2/LENS/atm/month_1/PSL/b.e21.BHISTsmbb.f09_g17.LE2-1301.020.cam.h0.PSL.190001-190912.nc', '/project/mojave/cesm2/LENS/atm/month_1/PSL/b.e21.BHISTsmbb.f09_g17.LE2-1301.020.cam.h0.PSL.191001-191912.nc', '/project/mojave/cesm2/LENS/atm/month_1/PSL/b.e21.BHISTsmbb.f09_g17.LE2-1301.020.cam.h0.PSL.192001-192912.nc', '/project/mojave/cesm2/LENS/atm/month_1/PSL/b.e21.BHISTsmbb.f09_g17.LE2-1301.020.cam.h0.PSL.193001-193912.nc', 

/project/cas/asphilli/anaconda3/lib/python3.8/site-packages/xarray/conventions.py:512: SerializationWarning: variable 'psl' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(


<xarray.Dataset>
Dimensions:    (lat: 192, lon: 288, nbnd: 2, time: 3012)
Coordinates:
  * lat        (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon        (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * time       (time) object 1850-01-15 12:00:00 ... 2100-12-15 12:00:00
Dimensions without coordinates: nbnd
Data variables:
    psl        (time, lat, lon) float32 dask.array<chunksize=(600, 192, 288), meta=np.ndarray>
    time_bnds  (time, nbnd) object dask.array<chunksize=(600, 2), meta=np.ndarray>
    lat_bnds   (time, lat, nbnd) float64 dask.array<chunksize=(600, 192, 2), meta=np.ndarray>
    lon_bnds   (time, lon, nbnd) float64 dask.array<chunksize=(600, 288, 2), meta=np.ndarray>
Attributes: (12/45)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   674885.0
    branch_time_in_parent:  306600.0
    case_id:                24
    ...                   

/project/cas/asphilli/anaconda3/lib/python3.8/site-packages/xarray/conventions.py:512: SerializationWarning: variable 'psl' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(


<xarray.Dataset>
Dimensions:    (lat: 192, lon: 288, nbnd: 2, time: 1980)
Coordinates:
  * lat        (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon        (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * time       (time) object 1850-01-15 12:00:00 ... 2014-12-15 12:00:00
Dimensions without coordinates: nbnd
Data variables:
    psl        (time, lat, lon) float32 dask.array<chunksize=(1980, 192, 288), meta=np.ndarray>
    time_bnds  (time, nbnd) object dask.array<chunksize=(1980, 2), meta=np.ndarray>
    lat_bnds   (lat, nbnd) float32 dask.array<chunksize=(192, 2), meta=np.ndarray>
    lon_bnds   (lon, nbnd) float32 dask.array<chunksize=(288, 2), meta=np.ndarray>
Attributes: (12/45)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    case_id:                18
    cesm_casename:          b.e21.BHIST.f09_g17.CMIP6-historical.004
    contact:                cesm_cmip6@ucar.edu
    creation_date:          2019-01-18T1

/project/cas/asphilli/anaconda3/lib/python3.8/site-packages/xarray/conventions.py:512: SerializationWarning: variable 'psl' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(


<xarray.Dataset>
Dimensions:    (lat: 192, lon: 288, nbnd: 2, time: 1980)
Coordinates:
  * lat        (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon        (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * time       (time) object 1850-01-15 12:00:00 ... 2014-12-15 12:00:00
Dimensions without coordinates: nbnd
Data variables:
    psl        (time, lat, lon) float32 dask.array<chunksize=(1980, 192, 288), meta=np.ndarray>
    time_bnds  (time, nbnd) object dask.array<chunksize=(1980, 2), meta=np.ndarray>
    lat_bnds   (lat, nbnd) float32 dask.array<chunksize=(192, 2), meta=np.ndarray>
    lon_bnds   (lon, nbnd) float32 dask.array<chunksize=(288, 2), meta=np.ndarray>
Attributes: (12/45)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    case_id:                19
    cesm_casename:          b.e21.BHIST.f09_g17.CMIP6-historical.005
    contact:                cesm_cmip6@ucar.edu
    creation_date:          2019-01-18T1

In [ ]:

#   Testing window

#a = np.array([[1, np.nan], [3, 4]])
a = xr.DataArray([[1, np.nan], [3, 4]],attrs={'_FillValue' : -999})
#a.attrs = {'_FillValue':-999} 
print(a.mean())
#print(a)

np.nan

In [ ]:
print(mean([-1.0, 2.5, 3.25, 5.75]))

In [ ]:
a = np.array([1, 2, 3, 4, 5, 6])
b = a[:1]
print(b)